In [1]:
import pandas as pd
df = pd.read_csv('sales.csv', usecols=["order_id", "order_date", "payment_method"])
df.drop_duplicates(subset=["order_id"], ignore_index=True, inplace=True)


/tmp/ipykernel_20656/47571307.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('sales.csv', usecols=["order_id", "order_date", "payment_method"])


In [2]:
df.head()

,order_id,order_date,payment_method
0,100354678,2020-10-01,cod
1,100354680,2020-10-01,cod
2,100367357,2020-11-13,cod
3,100367360,2020-11-13,cod
4,100354677,2020-10-01,Payaxis


In [3]:
# Apply the condition
df['payment_method'] = df['payment_method'].apply(lambda x: 'Cash on delivery' if x in ['cod', 'cashatdoorstep'] else 'Online payment')

In [4]:
df.head()

,order_id,order_date,payment_method
0,100354678,2020-10-01,Cash on delivery
1,100354680,2020-10-01,Cash on delivery
2,100367357,2020-11-13,Cash on delivery
3,100367360,2020-11-13,Cash on delivery
4,100354677,2020-10-01,Online payment


In [5]:
# Convert 'order_date' to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Add a new column 'week' for grouping by week
df['week'] = df['order_date'].dt.to_period('W')

# Group by 'week' and aggregate counts for different payment methods
result = df.groupby('week').agg(
    total_orders=('order_id', 'count'),
    cash_on_delivery_orders=('payment_method', lambda x: (x == 'Cash on delivery').sum()),
    online_payment_orders=('payment_method', lambda x: (x == 'Online payment').sum())
).reset_index()


In [6]:
result.head()

,week,total_orders,cash_on_delivery_orders,online_payment_orders
0,2020-09-28/2020-10-04,906,533,373
1,2020-10-05/2020-10-11,1265,775,490
2,2020-10-12/2020-10-18,1474,891,583
3,2020-10-19/2020-10-25,1438,814,624
4,2020-10-26/2020-11-01,1456,799,657


In [7]:
result["percent_cash_on_delivery_orders"] = result["cash_on_delivery_orders"]/result["total_orders"]
result["percent_online_payment_orders"] = result["online_payment_orders"]/result["total_orders"]

In [8]:
result.to_csv("ouput/payment_trend.csv", index=False)
